<a href="https://colab.research.google.com/github/Sauhardya007/Scrabble_analysis.ipynb/blob/main/Scrabble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#from tensorflow.keras.preprocessing.text import Tokenizer
#tokenizer=Tokenizer(oov_token="<nothing>",char_level=False)

In [ ]:
pip install kaggle

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/sentiment140


In [ ]:
data_path = "/root/.cache/kagglehub/datasets/kazanova/sentiment140/versions/2/training.1600000.processed.noemoticon.csv"

In [ ]:
df=pd.read_csv(data_path,encoding='latin-1',header=None)
df.columns = ["target", "ids", "date", "flag", "user", "text"]
#df.head(3)

In [ ]:
df["sentiment"] = df["target"].map({0: "negative", 1: "positive"})
#df.head(10)

In [ ]:
pip install pyspellchecker


In [ ]:
#data_preprocessing
import re
from spellchecker import SpellChecker
spell=SpellChecker()
chat_dict = {
    "u": "you",
    "ur": "your",
    "r": "are",
    "lol": "laugh out loud",
    "lmao": "laughing my ass off",
    "rofl": "rolling on the floor laughing",
    "brb": "be right back",
    "btw": "by the way",
    "omg": "oh my god",
    "idk": "I don't know",
    "idc": "I don't care",
    "imo": "in my opinion",
    "imho": "in my humble opinion",
    "ttyl": "talk to you later",
    "smh": "shaking my head",
    "fyi": "for your information",
    "np": "no problem",
    "thx": "thanks",
    "ty": "thank you",
    "yw": "you’re welcome",
    "w8": "wait",
    "b4": "before",
    "gr8": "great",
    "cya": "see you",
    "omw": "on my way",
    "asap": "as soon as possible",
    "nvm": "never mind",
    "bc": "because",
    "cu": "see you",
    "msg": "message",
    "pls": "please",
    "plz": "please",
    "ya": "yeah",
    "tho": "though",
    "dm": "direct message",
    "ikr": "I know right",
    "jk": "just kidding",
    "bff": "best friends forever",
    "wtf": "what the f***",
    "wth": "what the hell",
    "afk": "away from keyboard",
    "gg": "good game",
    "tf": "the f***",
    "bday": "birthday",
    "hbd": "happy birthday",
    "xoxo": "hugs and kisses",
    "aight": "alright",
    "atm": "at the moment",
    "bbl": "be back later",
    "bfn": "bye for now",
    "cba": "can't be arsed",
    "cuz": "because",
    "da": "the",
    "dunno": "don't know",
    "fomo": "fear of missing out",
    "ftw": "for the win",
    "g2g": "got to go",
    "gonna": "going to",
    "gotcha": "got you",
    "hbu": "how about you",
    "hmu": "hit me up",
    "howdy": "hello",
    "iirc": "if I recall correctly",
    "irl": "in real life",
    "jus": "just",
    "k": "okay",
    "kewl": "cool",
    "kinda": "kind of",
    "lemme": "let me",
    "luv": "love",
    "nah": "no",
    "naw": "no",
    "ne1": "anyone",
    "nite": "night",
    "nothin": "nothing",
    "nvm": "never mind",
    "oic": "oh I see",
    "omfg": "oh my f***ing god",
    "oop": "oops",
    "pics": "pictures",
    "prob": "problem",
    "prolly": "probably",
    "rn": "right now",
    "sec": "second",
    "sup": "what's up",
    "tbh": "to be honest",
    "tho": "though",
    "thx": "thanks",
    "tmrw": "tomorrow",
    "tryna": "trying to",
    "tysm": "thank you so much",
    "u2": "you too",
    "wassup": "what's up",
    "wbu": "what about you",
    "wdym": "what do you mean",
    "wru": "where are you",
    "wyd": "what are you doing",
    "ya": "you",
    "yo": "hey",
    "yolo": "you only live once",
    "yt": "youtube"}

def clean_text(text):
  text=text.lower()
  text=re.sub(r"http\S+|www\S+|https\S+","",text,flags=re.MULTILINE)
  text=re.sub(r"\@\w+|\#","",text)
  text=re.compile(r"<.*?>").sub(r"",text)
  text = re.sub(r"[^a-zA-Z\s]", "", text)
  text = re.sub(r"\s+", " ", text).strip()
  emoji_pattern = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags
    u"\U00002700-\U000027BF"  # dingbats
    u"\U0001F900-\U0001F9FF"  # supplemental symbols
    u"\U00002600-\U000026FF"  # miscellaneous symbols
    u"\U0001FA70-\U0001FAFF"  # extended pictographs
    u"\U000025A0-\U000025FF"  # geometric shapes
    "]+", flags=re.UNICODE)
  text = emoji_pattern.sub(r"", text)

  text =([chat_dict[word] if word in chat_dict else word for word in text.split()])
  #corrected_words = [
  #    spell.correction(word) if len(word) > 2 and spell.correction(word) is not None else word
  #    for word in text
  #]

  #return " ".join(corrected_words)
  return " ".join(text)
df['text']=df['text'].apply(clean_text)


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
max_vocab=30000
tokenizer = Tokenizer(num_words=max_vocab, oov_token="<nothing>")
tokenizer.fit_on_texts(df['text'])
sequences=tokenizer.texts_to_sequences(df['text'])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping



In [ ]:

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(sequences,df['sentiment'],test_size=0.2,random_state=40)

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=max_vocab,output_dim=50))
model.add(LSTM(128,return_sequences=True,kernel_regularizer=regularizers.l2(0.01)))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2,activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# Pad the sequences after the train-test split
max1=250
X_train = pad_sequences(X_train,maxlen=max1,padding='post',truncating='post')
X_test = pad_sequences(X_test,maxlen=max1,padding='post',truncating='post')
label_map = {'negative': 0, 'positive': 1}

X_train = np.array(X_train[:20000])
X_test = np.array(X_test[:20000])
y_train = np.array(y_train[:20000])
y_test = np.array(y_test[:20000])
y_train_num = np.array([label_map[x] for x in y_train])
y_test_num = np.array([label_map[x] for x in y_test])
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
model.fit(X_train,y_train_num,epochs=15,validation_data=(X_test,y_test_num),callbacks=[early_stop])

Epoch 1/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 39ms/step - accuracy: 0.7876 - loss: 0.4785 - val_accuracy: 0.7934 - val_loss: 0.4549
Epoch 2/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 35s 30ms/step - accuracy: 0.8221 - loss: 0.4243 - val_accuracy: 0.7907 - val_loss: 0.4602
Epoch 3/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.8515 - loss: 0.3732 - val_accuracy: 0.7829 - val_loss: 0.4839
Epoch 4/15
625/625 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - accuracy: 0.8749 - loss: 0.3310 - val_accuracy: 0.7720 - val_loss: 0.5278


In [ ]:
!pip install snscrape


In [ ]:

#!pip install --upgrade certifi

In [ ]:
"""
import snscrape.modules.twitter as sntwitter
search_query = "Scrabble"
num_tweets = 10
scrabble_texts = []
print(f"Scraping {num_tweets} tweets about '{search_query}'...")
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(search_query).get_items()):
    if i >= num_tweets:
        break
    scrabble_texts.append(tweet.rawContent)
  """

In [ ]:


# --- 1. Your Provided Data ---
# These are the example sentences you provided.
positive_examples = [
    "I love playing Scrabble with friends; it’s so much fun and educational!",
    "Scrabble keeps my mind sharp and improves my vocabulary.",
    "Winning a tough game of Scrabble feels incredibly satisfying.",
    "I enjoy family game night more because of Scrabble—it’s my favorite!",
    "Scrabble is a classic board game that never gets boring.",
    "Playing Scrabble always challenges me in a fun way.",
    "I love discovering new words while playing Scrabble.",
    "Scrabble night is the highlight of my week.",
    "Competing in Scrabble tournaments is exciting and rewarding.",
    "Scrabble brings friends and family together."
]

negative_examples = [
    "I hate losing at Scrabble; it’s so frustrating.",
    "Scrabble can be boring when the letters are all vowels.",
    "Playing Scrabble online is slow and full of glitches.",
    "I get annoyed when my opponents cheat in Scrabble.",
    "Scrabble is stressful when I can’t think of any words.",
    "Scrabble games sometimes take forever to finish.",
    "I get frustrated when I can’t find good words in Scrabble.",
    "Scrabble can feel unfair if luck of the draw is bad.",
    "Losing repeatedly at Scrabble makes me not want to play.",
    "Scrabble can be overwhelming with complex rules and scoring."
]

# --- 2. Combine all examples and create corresponding labels ---
# We'll put all the sentences into a single list and create a label for each one.
# 0 = Positive, 1 = Negative, 2 = Neutral
all_sentences = positive_examples + negative_examples
all_labels = ([1] * len(positive_examples) +
              [0] * len(negative_examples))
sequences=tokenizer.texts_to_sequences(all_sentences)
padded_sequences=pad_sequences(sequences,maxlen=max1,padding='pre',truncating='post')


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
all_labels

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
predicted=model.predict(padded_sequences)
np.argmax(predicted,axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step


array([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

In [ ]:
!git config --global user.email "sauhardyah@gmail.com"